<a href="https://colab.research.google.com/github/dkjgA893274/AI-simple-app/blob/main/section_1/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなチャットボット
LangChainとStreamlitを使い、チャットボットを構築します。  

## ライブラリのインストール
langchain-coreとlangchain-openaiをインストールします。  
また、UIを構築するための「Streamlit」、およびアプリの動作の確認に使用する「ngrok」をインストールします。  

In [ ]:
!pip install -U "streamlit>=1.35" "pandas==2.2.2" "protobuf>=5.29,<6" pyngrok langchain langchain-openai

## チャットボットのコード
`%%writefile`のマジックコマンドを使って、チャットボットのコードを「app.py」に書き込みます。  
Streamlitでは、値を保持するために「st.session_state」を使います。  
https://docs.streamlit.io/library/api-reference/session-state

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み

# ========================================
# 必要なライブラリのインポート
# ========================================
import streamlit as st  # Webアプリケーション作成用
from langchain_openai import ChatOpenAI  # OpenAIのチャットモデル
from langchain.schema import HumanMessage, SystemMessage  # メッセージの型定義
import os  # 環境変数操作用
import secret_keys  # 外部ファイルにAPI keyを保存（セキュリティのため）

# OpenAI APIキーを環境変数に設定
os.environ["OPENAI_API_KEY"] = secret_keys.openai_api_key

# GPT-3.5-turboモデルを使用してチャットインスタンスを作成
chat = ChatOpenAI(model="gpt-3.5-turbo")

# ========================================
# セッション状態の初期化
# ========================================
# st.session_stateを使いメッセージのやりとりを保存
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        SystemMessage(
            content="あなたは優秀なアシスタントAIです。"
        )
    ]

# ========================================
# 関数定義
# ========================================

# 会話履歴をクリアする関数
def clear_chat():
    """
    会話履歴をリセットして、新しい会話を開始できるようにする
    システムメッセージのみを残して、ユーザーとAIの会話履歴を削除
    """
    st.session_state["messages"] = [
        SystemMessage(
            content="あなたは優秀なアシスタントAIです。"
        )
    ]

# LLMとやりとりする関数
def communicate():
    """
    ユーザーの入力を受け取り、AIに送信して応答を取得する
    会話履歴にユーザーメッセージとAIの応答を追加
    """
    # 現在の会話履歴を取得
    messages = st.session_state["messages"]

    # ユーザーの入力をHumanMessageオブジェクトに変換
    user_message = HumanMessage(
        content=st.session_state["user_input"]
    )

    # 会話履歴にユーザーメッセージを追加
    messages.append(user_message)

    # AIにメッセージを送信して応答を取得
    response = chat(messages)

    # AIの応答を会話履歴に追加
    messages.append(response)

    st.session_state["user_input"] = ""  # 入力欄を消去

# ========================================
# ユーザーインターフェイスの構築
# ========================================

# アプリのタイトルと説明
st.title("My AI Assistant")
st.write("LangChainを使ったチャットボットです。")

# ========================================
# 入力エリア（テキスト入力 + クリアボタン）
# ========================================
# レイアウトを2列に分割（入力欄:クリアボタン = 3:1の比率）
col1, col2 = st.columns([3, 1])

with col1:
    # テキスト入力欄（Enterキーまたはボタンクリックで送信）
    user_input = st.text_input(
        "メッセージを入力してください。",
        key="user_input",
        on_change=communicate  # 入力時に自動的にcommunicate関数を実行
    )

with col2:
    # クリアボタンを入力欄と同じ高さに配置するため、適切なスペースを追加
    st.markdown("<br>", unsafe_allow_html=True)  # HTMLの改行タグでより正確に位置調整
    # 会話履歴をクリアするボタン
    if st.button("🗑️ 会話をクリア", help="会話履歴をリセットします"):
        clear_chat()  # 会話履歴をリセット
        st.rerun()    # ページを再読み込みしてUIを更新

# ========================================
# 会話履歴の表示
# ========================================
# メッセージが存在する場合のみ表示
if st.session_state["messages"]:
    messages = st.session_state["messages"]

    # 会話履歴を逆順で表示（最新のメッセージが上に来る）
    for message in reversed(messages[1:]):  # システムメッセージ（インデックス0）は除外
        # メッセージの送信者に応じてアイコンを設定
        speaker = "🙂"  # デフォルトはユーザー
        if message.type == "ai":
            speaker = "🤖"  # AIの場合はロボットアイコン

        # メッセージを表示（アイコン + 内容）
        st.write(speaker + ": " + message.content)

## OpenAIのAPI keyを設定
ChatGPT APIを使用するために必要な「API key」を設定します。  
`%%writefile`のマジックコマンドを使って、API keyを設定するコードを「secret_keys.py」に書き込みます。  
以下のコードの、  
`openai_api_key = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
ChatGPTのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [ ]:
%%writefile secret_keys.py

openai_api_key = "Your API key"

API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## ngrokのAuthtokenを設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [ ]:
!ngrok authtoken YourAuthtoken

ngrokのAuthtokenも、他者に知られないように慎重に扱ってください。

## アプリの起動
streamlitの`run`コマンドでアプリを起動します。


In [ ]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [ ]:
from pyngrok import ngrok

ngrok.kill()  # プロセスの修了
url = ngrok.connect("8501").public_url  # 接続

## 動作の確認

URLのhttpの部分をhttpsに変換する関数を設定します。

In [ ]:
def convert_http_to_https(url):
    if url.startswith("http://"):
        url = url.replace("http://", "https://", 1)
    return url

変換したurlを表示し、リンク先でチャットボットが動作することを確認します。

In [ ]:
print(convert_http_to_https(url))

チャットボットの動作確認後、OpenAIのサイトでAPIの使用量を確認してみましょう。  
https://platform.openai.com/account/usage